
# CRIM tools and their use in a pedagogical context: in search of soggetti in Palestrina’s *missa Veni sponsa Christi* (1599)
## Gabriele Taschetti, Marina Toffetti
### Supplement to the essay

This notebook, edited by Gabriele Taschetti, includes some of the code used to prepare the article "CRIM tools and their use in a pedagogical context: in search of soggetti in Palestrina’s *missa Veni sponsa Christi* (1599)" by Gabriele Taschetti and Marina Toffetti.

## A. Import Intervals and Other Code


In [ ]:
import IPython
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import glob as glob
audio_style = "<style>audio { margin-left: 0px; width: 960px; }</style>"
display(HTML(audio_style))


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

## B. Search in the corpus

### Preset search (preset corpus; kind='d', n=4, combineUnisons=True; preset soggetti)

In [ ]:
# ESEGUI LA CELLA e seleziona il soggetto
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_5.mei'])
func1 = ImportedPiece.notes
notes_df = corpus.batch(func=func1, kwargs={'combineUnisons': True}, metadata=False)
func2 = ImportedPiece.melodic
melodic_df = corpus.batch(func=func2, kwargs={'kind': 'd', 'end': False, 'df': notes_df}, metadata=False)
func3 = ImportedPiece.ngrams
ngrams_df = corpus.batch(func=func3, kwargs={'n': 4, 'df': melodic_df}, metadata=False)
func4 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func4, kwargs={'offset': False,'df': ngrams_df}, metadata=True)

mel_corpus = pd.concat(list_of_detail_index)
comp = mel_corpus.pop("Composer")
mel_corpus['Composer'] = comp
title = mel_corpus.pop("Title")
mel_corpus["Title"] = title
mel_corpus = mel_corpus.fillna('-')

def _convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out

@interact
def mel_ngram_search(soggetto=["-3, 3, 2, -2", "-3, 2, 2, -2", "-3, 3, -2, -2", "-2, 2, 3, -2"], df = fixed(mel_corpus)):
    df_no_tuple = df.applymap(_convertTuple)
    df_no_tuple.pop("Composer")
    df_no_tuple.pop("Title")
    df_no_tuple.insert(0, "Composer", df["Composer"])
    df_no_tuple.insert(1, "Title", df["Title"])
    filtered_ngrams = df_no_tuple[df_no_tuple.apply(lambda x: x.astype(str).str.contains(soggetto).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(soggetto, x) else "")


### Set your own parameters

In [ ]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_5.mei'])

### All occurrences

In [ ]:
# settings for ngrams, unisons and kind
combineUnisons = True
kind = 'd'
n = 3

In [ ]:
# ESEGUI LA CELLA e scrivi la tua ricerca
func1 = ImportedPiece.notes
notes_df = corpus.batch(func=func1, kwargs={'combineUnisons': combineUnisons}, metadata=False)
func2 = ImportedPiece.melodic
melodic_df = corpus.batch(func=func2, kwargs={'kind': kind, 'end': False, 'df': notes_df}, metadata=False)
func3 = ImportedPiece.ngrams
ngrams_df = corpus.batch(func=func3, kwargs={'n': n, 'df': melodic_df}, metadata=False)
func4 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func4, kwargs={'offset': False,'df': ngrams_df}, metadata=True)

mel_corpus = pd.concat(list_of_detail_index)
comp = mel_corpus.pop("Composer")
mel_corpus['Composer'] = comp
title = mel_corpus.pop("Title")
mel_corpus["Title"] = title
mel_corpus = mel_corpus.fillna('-')

def _convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out

@interact
def mel_ngram_search(my_search="", df = fixed(mel_corpus)):
    df_no_tuple = df.applymap(_convertTuple)
    df_no_tuple.pop("Composer")
    df_no_tuple.pop("Title")
    df_no_tuple.insert(0, "Composer", df["Composer"])
    df_no_tuple.insert(1, "Title", df["Title"])
    filtered_ngrams = df_no_tuple[df_no_tuple.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

### Entries only

In [ ]:
combineUnisons = True
kind = 'd'
n = 3

In [ ]:
func1 = ImportedPiece.notes
notes_df = corpus.batch(func=func1, kwargs={'combineUnisons': True}, metadata=False)
func2 = ImportedPiece.melodic
melodic_df = corpus.batch(func=func2, kwargs={'kind': 'd', 'end': False, 'df': notes_df}, metadata=False)
func3 = ImportedPiece.entries
entries_df = corpus.batch(func=func3, kwargs={'n': 4, 'df': melodic_df}, metadata=False)
func4 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func4, kwargs={'offset': False,'df': entries_df}, metadata=True)

mel_corpus = pd.concat(list_of_detail_index)
comp = mel_corpus.pop("Composer")
mel_corpus['Composer'] = comp
title = mel_corpus.pop("Title")
mel_corpus["Title"] = title
mel_corpus = mel_corpus.fillna('-')

def _convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out

@interact
def mel_ngram_search(my_search="", df = fixed(mel_corpus)):
    df_no_tuple = df.applymap(_convertTuple)
    df_no_tuple.pop("Composer")
    df_no_tuple.pop("Title")
    df_no_tuple.insert(0, "Composer", df["Composer"])
    df_no_tuple.insert(1, "Title", df["Title"])
    filtered_ngrams = df_no_tuple[df_no_tuple.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

### Check the music

In [ ]:
piece_list = ['CRIM_Model_0019.mei',
              'CRIM_Mass_0019_1.mei',
              'CRIM_Mass_0019_2.mei',
              'CRIM_Mass_0019_3.mei',
              'CRIM_Mass_0019_4.mei',
              'CRIM_Mass_0019_5.mei']
prefix = 'https://crimproject.org/mei/' 
mei_file = piece_list[5] # 0=model 1-5=mass movements (e. g. 2=Gloria)
url = prefix + mei_file
piece = importScore(url)
print(piece.metadata)
piece.verovioPrintExample(36, 41) # start measure, end measure

## C. Find the most recurrent entries in the model

In [ ]:
piece_list = ['CRIM_Model_0019',
              'CRIM_Mass_0019_1',
              'CRIM_Mass_0019_2',
              'CRIM_Mass_0019_3',
              'CRIM_Mass_0019_4',
              'CRIM_Mass_0019_5']

In [ ]:
# settings for ngrams, unisons and kind
n = 4
combineUnisons = True
kind = 'd'

# set the number of n-grams
n_of_ngrams = 4

In [ ]:
# select the model from the list
model = piece_list[0]
prefix = 'https://crimproject.org/mei/' 
# prefix = 'Music_Files/'
url = prefix + model + '.mei'
model = importScore(url)

# get the most frequent entries of the model
nr = model.notes(combineUnisons=combineUnisons)
mel = model.melodic(df=nr, kind=kind, end=False)
ng = model.ngrams(df=mel, n=n)
entries = model.entries(df=ng).stack().value_counts().to_frame().head(n_of_ngrams)
entries_chart = model.entries(df=ng).stack().value_counts().to_frame().head(n_of_ngrams)
entries_chart.columns = ['counts']
entries_list = entries_chart.index.tolist()
entries_chart

## D. Visualize the most recurrent entries of the model in the whole corpus
Visualize n-grams anywhere and only where they occur as entries (after a rest or section break)

In [ ]:
for piece in piece_list:
    prefix = 'https://crimproject.org/mei/'
    mei_file = piece
    url = prefix + mei_file + '.mei'
    audio_url = 'https://crimproject.org/static/mp3/' + mei_file + '.mp3'
    piece = importScore(url)
    nr = piece.notes(combineUnisons=combineUnisons)
    mel = piece.melodic(df=nr, kind=kind, compound=False, unit=0, end=False)
    mel_ngrams = piece.ngrams(df=mel, n=n)
    piece_entries = piece.entries(df=mel, n=n, anywhere=True)
    mel_ngrams_duration = piece.durations(df=mel, n=n, mask_df=piece_entries)
    print()
    print(piece.metadata)
    print('Anywhere')
    display(viz.plot_ngrams_heatmap(piece_entries, mel_ngrams_duration, selected_patterns=entries_list, voices=[]))
    piece_entries = piece.entries(df=mel, n=n, anywhere=False)
    mel_ngrams_duration = piece.durations(df=mel, n=n, mask_df=piece_entries)
    print('Entries only')
    display(viz.plot_ngrams_heatmap(piece_entries, mel_ngrams_duration, selected_patterns=entries_list, voices=[]))
    # load mp3 file (add/remove '#' at the beginning of the next line)
    display(IPython.display.Audio(audio_url))
    

### Check the music

In [ ]:
# select piece: 0=model 1-5=mass movements
check_piece = 1
# start number
start_offset = 426
# number of bars to display
bars = 4
# bars before the offset
bars_bf = 1

# load piece
prefix = 'https://crimproject.org/mei/' 
mei_file = piece_list[check_piece]
url = prefix + mei_file + '.mei'
piece = importScore(url)
# get offsets of the nGrams
mel = piece.melodic(kind=kind)
ng = piece.ngrams(df=mel, n=n)
final_ngs = piece.detailIndex(ng, offset=True)
# get measure number
idx = pd.IndexSlice
check = final_ngs.loc[idx[:,:,[start_offset]]]
index_list = check.index.tolist()
get_measure = index_list[0]
measure = int(*get_measure[:1])
# render example
start_ex = (measure - bars_bf)
end_ex = start_ex + (bars - 1)
print('Offset ' + str(start_offset) + ' is in measure ' + str(measure))
piece.verovioPrintExample(start_ex, end_ex)

## D. Search and visualize any list of n-grams in the corpus

In [ ]:
# settings for ngrams, unisons and kind
n = 4
combineUnisons = True
kind = 'd'

In [ ]:
list_1 = [('-3', '3', '2', '-2'),
            ('-3', '2', '2', '-2'),
            ('-3', '3', '-2', '-2'),
            ('-2', '2', '3', '-2')]
# list 2 = []

In [ ]:
selected_patterns = list_1

In [ ]:
for piece in piece_list:
    prefix = 'https://crimproject.org/mei/' 
    mei_file = piece
    url = prefix + mei_file + '.mei'
    piece = importScore(url)
    notes = piece.notes(combineUnisons=combineUnisons)
    notes_durs = piece.durations(df=notes, mask_df=notes)
    notes.replace("Rest", np.nan, inplace=True)
    nr = piece.notes(combineUnisons=combineUnisons)
    mel = piece.melodic(df=nr, kind=kind, compound=True, unit=0)
    mel_ngrams = piece.ngrams(df=mel, n=n)
    mel_ngrams_duration = piece.durations(df=mel, n=n, mask_df=mel_ngrams)
    print(piece.metadata)
    display(viz.plot_ngrams_heatmap(mel_ngrams, mel_ngrams_duration, selected_patterns=selected_patterns, voices=[]))
    #load audio
    #display(IPython.display.Audio('https://crimproject.org/static/mp3/' + mei_file + '.mp3'))

### Check the music

In [ ]:
# select piece: 0=model 1-5=mass movements
check_piece = 1
# start number
start_offset = 426
# number of bars to display
bars = 4
# bars before the offset
bars_bf = 1

# load piece
prefix = 'https://crimproject.org/mei/' 
mei_file = piece_list[check_piece]
url = prefix + mei_file + '.mei'
piece = importScore(url)
# get offsets of the nGrams
mel = piece.melodic(kind=kind)
ng = piece.ngrams(df=mel, n=n)
final_ngs = piece.detailIndex(ng, offset=True)
# get measure number
idx = pd.IndexSlice
check = final_ngs.loc[idx[:,:,[start_offset]]]
index_list = check.index.tolist()
get_measure = index_list[0]
measure = int(*get_measure[:1])
# render example
start_ex = (measure - bars_bf)
end_ex = start_ex + (bars - 1)
print('Offset ' + str(start_offset) + ' is in measure ' + str(measure))
piece.verovioPrintExample(start_ex, end_ex)

### Print any example

In [ ]:
prefix = 'https://crimproject.org/mei/' 
# select piece: 0=model 1-5=mass movements
mei_file = piece_list[0]
url = prefix + mei_file + '.mei'
piece = importScore(url)
# select first and last measure of the example
piece.verovioPrintExample(50, 54)